## Segmenting and Clustering Neighborhoods in Toronto #3

In [1]:
# importing the library and pulling the Wikipedia article
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [2]:
# importing the library for scraping the website content
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
# checking the content
#print(soup.prettify())

We will be exploring the HTML tags and extracting the table data. First, we have to locate the right table.

In [3]:
# Using BeautifulSoup to obtain the data that is in the table of postal codes
My_table = soup.find('table',{'class':'wikitable sortable'})
#My_table


As we can see, the tags we need are "tr" and "td". Let's start extracting the table data. Each table column will be stored in the separate list.

In [4]:
A=[]
B=[]
C=[]

for row in My_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True)[:-1])
        B.append(cells[1].find(text=True)[:-1])
        C.append(cells[2].find(text=True)[:-1])
#C

Next, we will transform the data into a pandas dataframe with three columns. 
After that we'll do some data cleanup by removing the obsolete entries.

In [5]:
import pandas as pd
df = pd.DataFrame()
df['PostalCodes'] = A
df['Borough'] = B
df['Neighbourhood'] = C

df = df[~df['Borough'].isin(['Not assigned'])]
df.reset_index(drop = True, inplace = True)

df.head()

,PostalCodes,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Next, we'll check the dataframe columns and data:

In [6]:
print(df.columns)
df.head(20)

Index(['PostalCodes', 'Borough', 'Neighbourhood'], dtype='object')


,PostalCodes,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Finally, let's check the dataframe dimensions.

In [7]:
df.shape

(103, 3)

Now that we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. We will use the csv file because it is provided.

In [8]:
#csv_path = "Downloads/Geospatial_Coordinates.csv"
csv_path = "C:/Users/UserName/Downloads/Geospatial_Coordinates.csv"
dfGeo = pd.read_csv(csv_path)
# rename column to align between two dataframes
dfGeo.rename(columns = {'Postal Code':'PostalCodes'}, inplace = True) 
dfGeo.head()

,PostalCodes,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We merge two frames on column "PostalCodes" in order to create a single table, then we sort it by postal code.

In [9]:
df = pd.merge(df, dfGeo, on="PostalCodes")
print(df.columns)
df.sort_index(axis = 1) 
df.head()

Index(['PostalCodes', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'], dtype='object')


,PostalCodes,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Therefore, we were able to use the provided csv file to get the geographical coordinates of the neighborhoods and create the dataframes. This concludes notebook 2.

Notebook #3. Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

In [10]:
neighborhoods = df.copy()
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [11]:
neighborhoods['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
York                 5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

Create a map of Toronto with neighborhoods superimposed on top.
### Please note: we will work with only boroughs that contain the word Toronto.

In [12]:
# removing all values not containing the word "Toronto"
neighborhoods = neighborhoods[~df['Borough'].isin(['Downtown Toronto','Central Toronto','East Toronto','West Toronto']) == False]
neighborhoods.reset_index(drop = True, inplace = True)

neighborhoods.head()

,PostalCodes,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [13]:
# as per above value_counts check, we are expecting to get 39 entries total
neighborhoods.shape

(39, 5)

Let's import necessary libraries and find the geograpical coordinates of Toronto in order to build the map.

In [20]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="TorontoON_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [21]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes 
#!pip install folium
# uncomment this line if you haven't completed the Foursquare API lab
import folium 
# map rendering library

# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

**Define Foursquare Credentials and Version**

In [22]:
CLIENT_ID = '0YOP1FXJVEUP5BOXUZG1FH3Y2EIWH04A5EYLAVRC2SUXR2XT' # your Foursquare ID
CLIENT_SECRET = 'BF5TKWMGU004CM3X2FVAAAAMOOHLCJ1PJUNO2ARGKFYBJW1A' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0YOP1FXJVEUP5BOXUZG1FH3Y2EIWH04A5EYLAVRC2SUXR2XT
CLIENT_SECRET:BF5TKWMGU004CM3X2FVAAAAMOOHLCJ1PJUNO2ARGKFYBJW1A


**Let's explore the first neighborhood in our dataframe**

In [24]:
neighborhoods.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

Get the neighborhood's latitude and longitude values.

In [25]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.
First, let's create the GET request URL (will be named "url").

In [26]:
search_query = ''
#search_query = 'Italian'
radius = 500
#print(search_query + ' .... OK!')
CLIENT_ID = '0YOP1FXJVEUP5BOXUZG1FH3Y2EIWH04A5EYLAVRC2SUXR2XT' # your Foursquare ID
CLIENT_SECRET = 'BF5TKWMGU004CM3X2FVAAAAMOOHLCJ1PJUNO2ARGKFYBJW1A' # your Foursquare Secret
ACCESS_TOKEN = '2SU305WMFH3PXFEAUD0D0H4H0KCA0NOC04CRBVIXHYHCYL03' # your FourSquare Access Token
VERSION = '20180605'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

print(url)

Your credentails:
CLIENT_ID: 0YOP1FXJVEUP5BOXUZG1FH3Y2EIWH04A5EYLAVRC2SUXR2XT
CLIENT_SECRET:BF5TKWMGU004CM3X2FVAAAAMOOHLCJ1PJUNO2ARGKFYBJW1A
https://api.foursquare.com/v2/venues/explore?&client_id=0YOP1FXJVEUP5BOXUZG1FH3Y2EIWH04A5EYLAVRC2SUXR2XT&client_secret=BF5TKWMGU004CM3X2FVAAAAMOOHLCJ1PJUNO2ARGKFYBJW1A&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100


Send the GET request and examine the resutls

In [27]:
import json # library to handle JSON files
import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from pandas import json_normalize

results = requests.get(url).json()
results['response']['groups'][0]['items'][0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '54ea41ad498e9a11e9e13308',
  'name': 'Roselle Desserts',
  'location': {'address': '362 King St E',
   'crossStreet': 'Trinity St',
   'lat': 43.653446723052674,
   'lng': -79.3620167174383,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.653446723052674,
     'lng': -79.3620167174383}],
   'distance': 143,
   'postalCode': 'M5A 1K9',
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['362 King St E (Trinity St)',
    'Toronto ON M5A 1K9',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d16a941735',
    'name': 'Bakery',
    'pluralName': 'Bakeries',
    'shortName': 'Bakery',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0

We know that all the information is in the items key. We will borrow the get_category_type function from the Foursquare lab.

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.

In [29]:
from pandas import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


And how many venues were returned by Foursquare?

In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

44 venues were returned by Foursquare.


**Explore Neighborhoods in Toronto**

Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues

In [32]:
neighborhoods = getNearbyVenues(neighborhoods['Neighbourhood'],nearby_venues['lat'],nearby_venues['lng'])
print("Done!")

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


Let's check the size of the resulting dataframe

In [33]:
print(neighborhoods.shape)
neighborhoods.head()

(1480, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.653447,-79.362017,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.653447,-79.362017,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.653447,-79.362017,Rooster Coffee,43.651900,-79.365609,Coffee Shop
3,"Regent Park, Harbourfront",43.653447,-79.362017,Starbucks,43.651613,-79.364917,Coffee Shop
4,"Regent Park, Harbourfront",43.653447,-79.362017,Distillery Sunday Market,43.650075,-79.361832,Farmers Market


Let's check how many venues were returned for each neighborhood

In [34]:
neighborhoods.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,49,49,49,49,49,49
"Brockton, Parkdale Village, Exhibition Place",42,42,42,42,42,42
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",28,28,28,28,28,28
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",41,41,41,41,41,41
Central Bay Street,49,49,49,49,49,49
Christie,41,41,41,41,41,41
Church and Wellesley,37,37,37,37,37,37
"Commerce Court, Victoria Hotel",32,32,32,32,32,32
Davisville,26,26,26,26,26,26


Let's find out how many unique categories can be curated from all the returned venues

In [35]:
print('There are {} uniques categories.'.format(len(neighborhoods['Venue Category'].unique())))

There are 77 uniques categories.


Analyze Each Neighborhood

In [37]:
# one hot encoding
toronto_onehot = pd.get_dummies(neighborhoods[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = neighborhoods['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Animal Shelter,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bakery,...,Skating Rink,Spa,Spanish Restaurant,Tech Startup,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Tram Station,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [38]:
toronto_onehot.shape

(1480, 78)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [39]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Animal Shelter,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bakery,...,Skating Rink,Spa,Spanish Restaurant,Tech Startup,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Tram Station,Yoga Studio
0,Berczy Park,0.000000,0.020408,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.061224,...,0.000000,0.020408,0.020408,0.020408,0.000000,0.061224,0.000000,0.000000,0.000000,0.020408
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,...,0.023810,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.023810
2,"Business reply mail Processing Centre, South C...",0.035714,0.000000,0.000000,0.000000,0.000000,0.035714,0.035714,0.000000,0.035714,...,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.024390,0.024390,0.000000,0.000000,0.024390,0.000000,0.000000,0.073171,...,0.000000,0.000000,0.024390,0.024390,0.000000,0.048780,0.000000,0.024390,0.024390,0.000000
4,Central Bay Street,0.000000,0.020408,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.061224,...,0.000000,0.020408,0.020408,0.020408,0.020408,0.061224,0.000000,0.000000,0.020408,0.020408
5,Christie,0.000000,0.024390,0.024390,0.000000,0.000000,0.024390,0.000000,0.000000,0.073171,...,0.000000,0.000000,0.024390,0.024390,0.000000,0.048780,0.000000,0.024390,0.024390,0.000000
6,Church and Wellesley,0.027027,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.027027,...,0.000000,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027
7,"Commerce Court, Victoria Hotel",0.031250,0.000000,0.000000,0.000000,0.000000,0.031250,0.031250,0.000000,0.000000,...,0.000000,0.031250,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,...,0.038462,0.038462,0.000000,0.000000,0.038462,0.038462,0.000000,0.000000,0.000000,0.038462
9,Davisville North,0.000000,0.024390,0.024390,0.000000,0.000000,0.000000,0.000000,0.000000,0.073171,...,0.000000,0.000000,0.024390,0.024390,0.000000,0.073171,0.000000,0.000000,0.024390,0.000000


Let's confirm the new size

In [40]:
toronto_grouped.shape

(39, 78)

Let's print each neighborhood along with the top 5 most common venues

In [41]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.18
1         Theater  0.06
2          Bakery  0.06
3  Breakfast Spot  0.04
4             Pub  0.04


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0         Coffee Shop  0.17
1      Breakfast Spot  0.07
2             Theater  0.07
3  Italian Restaurant  0.05
4           Pet Store  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
         venue  freq
0  Coffee Shop  0.21
1         Park  0.18
2         Café  0.07
3          Spa  0.04
4   Food Truck  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
           venue  freq
0    Coffee Shop  0.15
1         Bakery  0.07
2        Theater  0.05
3  Boat or Ferry  0.05
4      Gastropub  0.02


----Central Bay Street----
         venue  freq
0  Coffee Shop  0.16
1          Pub  0.06
2      Theater  0.06
3       Bakery  0.06
4  

**Let's put that into a pandas dataframe**

First, let's write a function to sort the venues in descending order.

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [52]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Theater,Bakery,Park,Breakfast Spot,Café,Pub,Yoga Studio,Electronics Store,Hotel
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Theater,Breakfast Spot,Italian Restaurant,Pet Store,Café,Yoga Studio,Farmers Market,Gym Pool,Gym / Fitness Center
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Park,Café,Beer Store,Fast Food Restaurant,Performing Arts Venue,Pet Store,Pharmacy,Pool,Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Bakery,Boat or Ferry,Theater,Gastropub,Historic Site,Breakfast Spot,Brewery,Café,Hotel
4,Central Bay Street,Coffee Shop,Park,Theater,Bakery,Pub,Breakfast Spot,Café,Historic Site,Mediterranean Restaurant,Bank
5,Christie,Coffee Shop,Bakery,Boat or Ferry,Theater,Gastropub,Historic Site,Breakfast Spot,Brewery,Café,Hotel
6,Church and Wellesley,Coffee Shop,Park,Food Truck,Pharmacy,Breakfast Spot,Café,Restaurant,Yoga Studio,History Museum,Gym / Fitness Center
7,"Commerce Court, Victoria Hotel",Coffee Shop,Park,Café,Breakfast Spot,Restaurant,Pharmacy,Animal Shelter,Pool,Fast Food Restaurant,Food Truck
8,Davisville,Coffee Shop,Italian Restaurant,Breakfast Spot,Yoga Studio,Bakery,Food Truck,Electronics Store,Discount Store,German Restaurant,Diner
9,Davisville North,Coffee Shop,Theater,Bakery,Gastropub,Historic Site,Breakfast Spot,Karaoke Bar,Brewery,Italian Restaurant,Burrito Place


## Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [53]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 0, 2, 4, 1, 4, 2, 2, 0, 4])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [54]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",43.653447,-79.362017,Roselle Desserts,43.653447,-79.362017,Bakery,4,Coffee Shop,Theater,Bakery,Pub,Breakfast Spot,Yoga Studio,Hotel,Historic Site,Gym Pool,Gym / Fitness Center
1,"Regent Park, Harbourfront",43.653447,-79.362017,Tandem Coffee,43.653559,-79.361809,Coffee Shop,4,Coffee Shop,Theater,Bakery,Pub,Breakfast Spot,Yoga Studio,Hotel,Historic Site,Gym Pool,Gym / Fitness Center
2,"Regent Park, Harbourfront",43.653447,-79.362017,Rooster Coffee,43.651900,-79.365609,Coffee Shop,4,Coffee Shop,Theater,Bakery,Pub,Breakfast Spot,Yoga Studio,Hotel,Historic Site,Gym Pool,Gym / Fitness Center
3,"Regent Park, Harbourfront",43.653447,-79.362017,Starbucks,43.651613,-79.364917,Coffee Shop,4,Coffee Shop,Theater,Bakery,Pub,Breakfast Spot,Yoga Studio,Hotel,Historic Site,Gym Pool,Gym / Fitness Center
4,"Regent Park, Harbourfront",43.653447,-79.362017,Distillery Sunday Market,43.650075,-79.361832,Farmers Market,4,Coffee Shop,Theater,Bakery,Pub,Breakfast Spot,Yoga Studio,Hotel,Historic Site,Gym Pool,Gym / Fitness Center


Finally, let's visualize the resulting clusters

In [56]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

#fix the NaN values in order to build the graph!!!
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].replace(np.nan, 0)

for lat, lon, poi, cluster in zip(toronto_merged['Neighborhood Latitude'], toronto_merged['Neighborhood Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
      
map_clusters

## Examine Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

**Cluster 1**

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
441,43.655675,-79.365873,Event Space,0,Coffee Shop,Italian Restaurant,Breakfast Spot,Yoga Studio,Pub,Food Truck,Electronics Store,Discount Store,Diner,Grocery Store
442,43.655675,-79.362017,Bakery,0,Coffee Shop,Italian Restaurant,Breakfast Spot,Yoga Studio,Pub,Food Truck,Electronics Store,Discount Store,Diner,Grocery Store
443,43.655675,-79.364503,Breakfast Spot,0,Coffee Shop,Italian Restaurant,Breakfast Spot,Yoga Studio,Pub,Food Truck,Electronics Store,Discount Store,Diner,Grocery Store
444,43.655675,-79.361809,Coffee Shop,0,Coffee Shop,Italian Restaurant,Breakfast Spot,Yoga Studio,Pub,Food Truck,Electronics Store,Discount Store,Diner,Grocery Store
445,43.655675,-79.365681,Thai Restaurant,0,Coffee Shop,Italian Restaurant,Breakfast Spot,Yoga Studio,Pub,Food Truck,Electronics Store,Discount Store,Diner,Grocery Store
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096,43.652756,-79.361222,Art Gallery,0,Coffee Shop,Theater,Italian Restaurant,Bakery,Breakfast Spot,Yoga Studio,Event Space,Gym Pool,Gym / Fitness Center,German Restaurant
1097,43.652756,-79.360073,Brewery,0,Coffee Shop,Theater,Italian Restaurant,Bakery,Breakfast Spot,Yoga Studio,Event Space,Gym Pool,Gym / Fitness Center,German Restaurant
1098,43.652756,-79.370115,Coffee Shop,0,Coffee Shop,Theater,Italian Restaurant,Bakery,Breakfast Spot,Yoga Studio,Event Space,Gym Pool,Gym / Fitness Center,German Restaurant
1099,43.652756,-79.369771,Skating Rink,0,Coffee Shop,Theater,Italian Restaurant,Bakery,Breakfast Spot,Yoga Studio,Event Space,Gym Pool,Gym / Fitness Center,German Restaurant


**Cluster 2**

In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
92,43.653249,-79.358008,Distribution Center,1,Coffee Shop,Bakery,Park,Pub,Café,Chocolate Shop,Historic Site,Hotel,Tram Station,Brewery
93,43.653249,-79.354390,Mediterranean Restaurant,1,Coffee Shop,Bakery,Park,Pub,Café,Chocolate Shop,Historic Site,Hotel,Tram Station,Brewery
94,43.653249,-79.357593,Performing Arts Venue,1,Coffee Shop,Bakery,Park,Pub,Café,Chocolate Shop,Historic Site,Hotel,Tram Station,Brewery
95,43.653249,-79.358127,Chocolate Shop,1,Coffee Shop,Bakery,Park,Pub,Café,Chocolate Shop,Historic Site,Hotel,Tram Station,Brewery
96,43.653249,-79.361809,Coffee Shop,1,Coffee Shop,Bakery,Park,Pub,Café,Chocolate Shop,Historic Site,Hotel,Tram Station,Brewery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1368,43.653081,-79.359702,Boutique,1,Coffee Shop,Café,Park,Pub,Bakery,Art Gallery,Electronics Store,Breakfast Spot,Italian Restaurant,Beer Store
1369,43.653081,-79.358785,Arts & Crafts Store,1,Coffee Shop,Café,Park,Pub,Bakery,Art Gallery,Electronics Store,Breakfast Spot,Italian Restaurant,Beer Store
1370,43.653081,-79.359012,Kids Store,1,Coffee Shop,Café,Park,Pub,Bakery,Art Gallery,Electronics Store,Breakfast Spot,Italian Restaurant,Beer Store
1371,43.653081,-79.363216,Bus Stop,1,Coffee Shop,Café,Park,Pub,Bakery,Art Gallery,Electronics Store,Breakfast Spot,Italian Restaurant,Beer Store


**Cluster 3**

In [62]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
627,43.658135,-79.359515,Coffee Shop,2,Coffee Shop,Park,Café,Breakfast Spot,Restaurant,Pharmacy,Animal Shelter,Pool,Fast Food Restaurant,Food Truck
628,43.658135,-79.356980,Restaurant,2,Coffee Shop,Park,Café,Breakfast Spot,Restaurant,Pharmacy,Animal Shelter,Pool,Fast Food Restaurant,Food Truck
629,43.658135,-79.361392,Pool,2,Coffee Shop,Park,Café,Breakfast Spot,Restaurant,Pharmacy,Animal Shelter,Pool,Fast Food Restaurant,Food Truck
630,43.658135,-79.358967,Pub,2,Coffee Shop,Park,Café,Breakfast Spot,Restaurant,Pharmacy,Animal Shelter,Pool,Fast Food Restaurant,Food Truck
631,43.658135,-79.356583,Animal Shelter,2,Coffee Shop,Park,Café,Breakfast Spot,Restaurant,Pharmacy,Animal Shelter,Pool,Fast Food Restaurant,Food Truck
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,43.657773,-79.359381,Café,2,Coffee Shop,Park,Café,Beer Store,Fast Food Restaurant,Performing Arts Venue,Pet Store,Pharmacy,Pool,Restaurant
1476,43.657773,-79.359871,Park,2,Coffee Shop,Park,Café,Beer Store,Fast Food Restaurant,Performing Arts Venue,Pet Store,Pharmacy,Pool,Restaurant
1477,43.657773,-79.352552,Bakery,2,Coffee Shop,Park,Café,Beer Store,Fast Food Restaurant,Performing Arts Venue,Pet Store,Pharmacy,Pool,Restaurant
1478,43.657773,-79.357553,Pet Store,2,Coffee Shop,Park,Café,Beer Store,Fast Food Restaurant,Performing Arts Venue,Pet Store,Pharmacy,Pool,Restaurant


**Cluster 4**

In [63]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
169,43.656369,-79.354806,Park,3,Coffee Shop,Park,Pub,Animal Shelter,Bank,Hotel,Café,Mediterranean Restaurant,Breakfast Spot,Gym / Fitness Center
170,43.656369,-79.356980,Restaurant,3,Coffee Shop,Park,Pub,Animal Shelter,Bank,Hotel,Café,Mediterranean Restaurant,Breakfast Spot,Gym / Fitness Center
171,43.656369,-79.356211,Park,3,Coffee Shop,Park,Pub,Animal Shelter,Bank,Hotel,Café,Mediterranean Restaurant,Breakfast Spot,Gym / Fitness Center
172,43.656369,-79.359515,Coffee Shop,3,Coffee Shop,Park,Pub,Animal Shelter,Bank,Hotel,Café,Mediterranean Restaurant,Breakfast Spot,Gym / Fitness Center
173,43.656369,-79.356583,Animal Shelter,3,Coffee Shop,Park,Pub,Animal Shelter,Bank,Hotel,Café,Mediterranean Restaurant,Breakfast Spot,Gym / Fitness Center
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1203,43.656739,-79.350955,Pizza Place,3,Coffee Shop,Park,Gym / Fitness Center,Pub,Distribution Center,Beer Store,Hotel,Café,Mediterranean Restaurant,Food Truck
1204,43.656739,-79.358670,Athletics & Sports,3,Coffee Shop,Park,Gym / Fitness Center,Pub,Distribution Center,Beer Store,Hotel,Café,Mediterranean Restaurant,Food Truck
1205,43.656739,-79.355867,Coffee Shop,3,Coffee Shop,Park,Gym / Fitness Center,Pub,Distribution Center,Beer Store,Hotel,Café,Mediterranean Restaurant,Food Truck
1206,43.656739,-79.357040,Hotel,3,Coffee Shop,Park,Gym / Fitness Center,Pub,Distribution Center,Beer Store,Hotel,Café,Mediterranean Restaurant,Food Truck


**Cluster 5**

In [64]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.653447,-79.362017,Bakery,4,Coffee Shop,Theater,Bakery,Pub,Breakfast Spot,Yoga Studio,Hotel,Historic Site,Gym Pool,Gym / Fitness Center
1,43.653447,-79.361809,Coffee Shop,4,Coffee Shop,Theater,Bakery,Pub,Breakfast Spot,Yoga Studio,Hotel,Historic Site,Gym Pool,Gym / Fitness Center
2,43.653447,-79.365609,Coffee Shop,4,Coffee Shop,Theater,Bakery,Pub,Breakfast Spot,Yoga Studio,Hotel,Historic Site,Gym Pool,Gym / Fitness Center
3,43.653447,-79.364917,Coffee Shop,4,Coffee Shop,Theater,Bakery,Pub,Breakfast Spot,Yoga Studio,Hotel,Historic Site,Gym Pool,Gym / Fitness Center
4,43.653447,-79.361832,Farmers Market,4,Coffee Shop,Theater,Bakery,Pub,Breakfast Spot,Yoga Studio,Hotel,Historic Site,Gym Pool,Gym / Fitness Center
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1410,43.650022,-79.357538,Antique Shop,4,Coffee Shop,Theater,Bakery,Boat or Ferry,Gastropub,Historic Site,Breakfast Spot,Brewery,Italian Restaurant,Hotel
1411,43.650022,-79.356490,Tram Station,4,Coffee Shop,Theater,Bakery,Boat or Ferry,Gastropub,Historic Site,Breakfast Spot,Brewery,Italian Restaurant,Hotel
1412,43.650022,-79.360215,Boat or Ferry,4,Coffee Shop,Theater,Bakery,Boat or Ferry,Gastropub,Historic Site,Breakfast Spot,Brewery,Italian Restaurant,Hotel
1413,43.650022,-79.357040,Hotel,4,Coffee Shop,Theater,Bakery,Boat or Ferry,Gastropub,Historic Site,Breakfast Spot,Brewery,Italian Restaurant,Hotel


This concludes notebook #3